In [5]:
import torch.nn as nn
import torch

n_points = 100

model = nn.Sequential(
    nn.Linear(in_features = n_points, out_features= 256),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(128, n_points),
)

In [6]:
loss = nn.MSELoss()

In [3]:
n_points = 200

epochs = 10000
lr = 0.007

for i in range(epochs):

  y = model(torch.from_numpy(V_train).float())

  losss = loss(y, torch.from_numpy(wave_train).float())

  model.zero_grad()

  losss.backward()

  with torch.no_grad():

    for param in model.parameters():
      param -= lr * param.grad

  if i%100 == 0:
    print(f'Epoch {i}/{epochs} Loss {losss}')

In [9]:
model = torch.load('../Models/first_model.pt')
model.eval()

Sequential(
  (0): Linear(in_features=100, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.1, inplace=False)
  (3): Linear(in_features=256, out_features=256, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.1, inplace=False)
  (6): Linear(in_features=256, out_features=128, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.1, inplace=False)
  (9): Linear(in_features=128, out_features=128, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.1, inplace=False)
  (12): Linear(in_features=128, out_features=100, bias=True)
)

In [10]:
predict = model(torch.from_numpy(V_test).float())

print("Test MSE: %f" % loss(predict, torch.from_numpy(wave_test).float()))
predict = predict.detach().numpy()

NameError: name 'V_test' is not defined